In [1]:
import gymnasium as gym
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
import numpy as np

env = gym.make("CartPole-v1")

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [10]:
n_observations = 4
n_actions = 2
n_hidden = 128


def create_q_model():
    observations = layers.Input(shape=(n_observations,))
    hidden = layers.Dense(n_hidden, activation='relu')(observations)
    action = layers.Dense(n_actions, activation='softmax')(hidden)
    
    return keras.Model(inputs=observations, outputs=action)

policy_network = create_q_model()
target_network = create_q_model()

In [16]:
state = np.array(env.reset()[0])

state_tensor = tf.convert_to_tensor(state)
state_tensor = tf.expand_dims(state_tensor, 0)

action_probs = policy_network(state_tensor, training=False)
action = tf.argmax(action_probs[0]).numpy()

1

In [26]:
action_history = []
rewards_history = []
state_history = []
state_next_history = []
done_history = []

epsilon = 1.0 
state = np.array(env.reset()[0])

for timestep in range(10000):

    if epsilon > np.random.rand(1)[0]:
        action = np.random.choice(2)
    else:
        state_tensor = tf.convert_to_tensor(state)
        state_tensor = tf.expand_dims(state_tensor, 0)
        action_probs = policy_network(state_tensor, training=False)
        action = tf.argmax(action_probs[0]).numpy()

    epsilon -= epsilon*0.99
    
    state_next, reward, done, _, _= env.step(action)
    
    action_history.append(action)
    rewards_history.append(reward)
    state_history.append(state)
    state_next_history.append(state_next)
    done_history.append(done)
        
    state_next = np.array(state_next)
    
    state = state_next

In [27]:
state

array([1.7808305e+05, 1.7961718e+03, 5.2074351e+03, 3.4877911e+01],
      dtype=float32)